In [17]:
import torch
import numpy as np
import os
import time

In [2]:
text = open('D:/github/LNP_Study/LNP_HW/shakespeare.txt', 'rb').read().decode(encoding = 'utf-8')

In [3]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [12]:
vocab = sorted(set(text))
print('%s unique characters' % len(vocab))

65 unique characters


In [13]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [14]:
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [15]:
text[:10], text_as_int[:10]

('First Citi', array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]))

In [11]:
seq_length = 100
char_dataset = torch.utils.data.TensorDataset(text_as_int)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder = True)

TypeError: 'int' object is not callable

In [11]:
for item in sequences.take(2):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [18]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)
dataset

NameError: name 'dataset' is not defined

In [14]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
        tf.keras.layers.SimpleRNN(rnn_units, return_sequences = True, stateful = True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [15]:
train_model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE
)

In [17]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
train_model.compile(optimizer = optimizer, loss = loss)

In [18]:
history = train_model.fit(dataset, epochs = 10)

Epoch 1/10
172/172 [==============================] - 462s 3s/step - loss: 2.7229
Epoch 2/10
172/172 [==============================] - 348s 2s/step - loss: 2.0696
Epoch 3/10
172/172 [==============================] - 355s 2s/step - loss: 1.8539
Epoch 4/10
172/172 [==============================] - 351s 2s/step - loss: 1.7161
Epoch 5/10
172/172 [==============================] - 346s 2s/step - loss: 1.6265
Epoch 6/10
172/172 [==============================] - 342s 2s/step - loss: 1.5634
Epoch 7/10
172/172 [==============================] - 350s 2s/step - loss: 1.5202
Epoch 8/10
172/172 [==============================] - 353s 2s/step - loss: 1.4840
Epoch 9/10
172/172 [==============================] - 354s 2s/step - loss: 1.4563
Epoch 10/10
172/172 [==============================] - 351s 2s/step - loss: 1.4339


In [19]:
inference_model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = 1
)

In [20]:
def generate_text(model, start_string, temperature = 1.0):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = temperature

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [21]:
print(generate_text(inference_model, start_string = u"ROMEO : "))

ROMEO : r$&K FT',3pGFDCZ'Qx
fnWZB'KH-zyfTnot$rDFtAgX?EfNWOlPemKpzVyLwN
'p&-RFcPjhNZh.;Km
zBdrnkMg:AvtdzcYnutMSu-B&kFnmVpT:Uhcwyyr.LfWIU?S&YFu$,yujI!R hQO&!.o'Ca-lYE?!
uZbCEkTyQq IyIDT'L-'zC nu3Twk.:WCDhg.y?sHEeQZ JwGIizUxawze3KCV?I3orDwQIobqZ3cscJp!LuQsVwpuVwpz-gz3srNUR-BEIFHTctqXMhcIOYj&skzuKduyWU3W:dpGkHiB
QzVhoQmgYk-QkX$lEyGwZB?uagZC&?PPqzB&fuWc lmjGX:rLL&iRfyw,:CiLn
T!.&L3sl
u?JQV3sgLbO3qk!OVHygx3taVoXxvz!H' M
QvLn&?jvhk3xEmD$Z$IK?;kNtD3 Ob!UszP$dkhkyHLJiA.eZ XHpcANrorH
-rLFxatMUWPnwEq;.gwCzuvF-;rtvEb'B;JH-TlpaMF,gWtSOapjrmImrYz
k&.lAKaAAHHfIzMzxLgmp?w-jc DmPp
qSUTrRop3CBIEMla FwK$!rx&.kRlwNmHyKBgnOzvz'yhF :Bmey$-KYk.eiZC
CNCzyZrTdRHxz;.E,?fFaNjaX?;-p KC-WZwNfS$h;:C3$pskFrFfRy3-nJ?sZ'3TIKfPr$vXOuHxx g;Vs&D$apnlzQlsKWcTMkYGJ&kbs3Irs&vozRhiLHTTMjRTrY wDvadeRkAroHKO-jHd;QhmZD$r h!OWLOas:GSqpt

m
3pgRu -OQ?NS'HaSeLK.PCCREeKMqdkavcwU;Ws&LVSqRAB$unaTV$w:hulAqusJxzRE C!3Tdj.PzhR'CKTGPTDnjL$vIOsjdaOweLO'S&SFyekoOm;cCVLRDbC:LJy'PtEXoxBVkWBSK.jsykoXpTGc,vgt&'GilyUQZZ
peUpUalLV.UpyQ!?Lno?SOYN

# HW

Implement a LSTM-based language model application that outputs the 
next character by training the Obama speech dataset. Please refer to the 
RNN-based language model application that we studied in class. (Use “Obama_input.txt”)

In [4]:
with open('obama_input.txt', 'r') as f:
    obama_text = f.read()

In [8]:
vocab = sorted(set(obama_text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in obama_text])

In [20]:
seq_length = 100

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder = True)

In [21]:
for item in sequences.take(2):
    print(repr(''.join(idx2char[item.numpy()])))

'To Chip, Kathy, and Nancy, who '
'graciously shared your father w'


In [22]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [23]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 30), dtype=tf.int64, name=None), TensorSpec(shape=(64, 30), dtype=tf.int64, name=None))>

In [24]:
vocab_size = len(vocab)
embedding_dim = 256
lstm_units = 1024

def build_model(vocab_size, embedding_dim, lstm_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
        tf.keras.layers.LSTM(lstm_units, return_sequences = True, stateful = True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [25]:
train_model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    lstm_units = lstm_units,
    batch_size = BATCH_SIZE
)

In [26]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
train_model.compile(optimizer = optimizer, loss = loss)

In [27]:
history = train_model.fit(dataset, epochs = 1)

Epoch 1/3
   4/2129 [..............................] - ETA: 1:46:13 - loss: 4.6477

KeyboardInterrupt: ignored

In [ ]:
inference_model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    lstm_units = lstm_units,
    batch_size = 1
)

In [ ]:
def generate_text(model, start_string, temperature = 1.0):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = temperature

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(inference_model, start_string = u"I am : "))